In [3]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay

In [4]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [11]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [7]:
# load json and create model
json_file = open('models/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("models/emotion_model.h5")
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [8]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=20,        validation_data=validation_generator,
        validation_steps=7178 // 64)


Epoch 1/20
448/448 [==============================] - 481s 1s/step - loss: 0.5090 - accuracy: 0.8152 - val_loss: 1.1223 - val_accuracy: 0.6321
Epoch 2/20
448/448 [==============================] - 164s 367ms/step - loss: 0.4932 - accuracy: 0.8246 - val_loss: 1.1491 - val_accuracy: 0.6260
Epoch 3/20
448/448 [==============================] - 168s 374ms/step - loss: 0.4720 - accuracy: 0.8296 - val_loss: 1.1538 - val_accuracy: 0.6247
Epoch 4/20
448/448 [==============================] - 172s 384ms/step - loss: 0.4573 - accuracy: 0.8335 - val_loss: 1.1481 - val_accuracy: 0.6275
Epoch 5/20
448/448 [==============================] - 173s 386ms/step - loss: 0.4408 - accuracy: 0.8401 - val_loss: 1.1779 - val_accuracy: 0.6290
Epoch 6/20
448/448 [==============================] - 169s 377ms/step - loss: 0.4318 - accuracy: 0.8430 - val_loss: 1.1649 - val_accuracy: 0.6350
Epoch 7/20
448/448 [==============================] - 175s 391ms/step - loss: 0.4105 - accuracy: 0.8512 - val_loss: 1.1894 - va

In [10]:
emotion_model.evaluate(validation_generator)

113/113 [==============================] - 6s 54ms/step - loss: 1.2877 - accuracy: 0.6286


[1.287705421447754, 0.6285873651504517]

In [9]:
model_json = emotion_model.to_json()
with open("models/emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('models/emotion_model.h5')